This is the basic version of the notebook. This notebook assumes that you have READ the documentation and no support will be given.

# Preparation

In [ ]:
#@title #Install Fish-Diffusion
#@markdown The stuff you'll need for every other thing afterwards.
from IPython.display import clear_output
import os
!rm -rf /content/sample_data

#install python 3.10 and dev utils
#you may not need all the dev libraries, but I haven't tested which aren't necessary.
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils libpython3.10-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2

# install pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

#install colab's dependencies
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor protobuf

# link to the old google package
!ln -s /usr/local/lib/python3.7/dist-packages/google \
       /usr/local/lib/python3.10/dist-packages/google

!git clone https://github.com/fishaudio/fish-diffusion

!python -m pip install --upgrade pip wheel
!pip install -q --pre poetry

%cd fish-diffusion
!wget https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/hifigan_44k.zip
!unzip -o hifigan_44k.zip -d checkpoints
!poetry install
!pip install tensorboard<2.9,>=2.8 --quiet
!python -m pip install .
count = -1

You are now expected to extract the wavs

In [ ]:
#@title #Pre-processing
#@markdown This just converts your data into mel spectograms

!python tools/preprocessing/extract_features.py --config configs/svc_hubert_soft.py --path dataset --clean

!python tools/preprocessing/generate_stats.py --input-dir dataset/train --output-file dataset/stats.json

In [ ]:
#@title #Tensorboard (run before step 5)
#@markdown Shows training progress, go to the top right corner to set it to update the logs.

import datetime, os
count += 1
%load_ext tensorboard
%tensorboard --load_fast=true --reload_interval=1 --reload_multifile=true --logdir=logs/DiffSVC/version_{count}

In [ ]:
#@title #Training
#@markdown Trains your model. Logs are shown in tensorboard, however

os.environ['PYTHONPATH']='.'

!CUDA_VISIBLE_DEVICES=0 python train.py --config configs/svc_hubert_soft.py --tensorboard